## Test different ideas for generating _future_ Boundary Conditions (BCs):
- SST
- Ice concentration

### Needed modules

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

%matplotlib inline

## To apply land-sea mask

In [ ]:
land_sea_mask = xr.open_dataset("/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/gen_daily_clim_data/data/geos_fp_bcs_land_sea_mask.nc")
my_mask = land_sea_mask.land_mask.values

### Local functions

In [ ]:
# file names: real data and daily climatology
def get_files_names(dates, data_path, file_pref, clim=False, file_suff=".nc"):
   files_to_read = []
   for idate in dates:

     if (clim==False): # real data
       ff = data_path + str(idate.year) + "/" +\
            file_pref + str(idate.year) + str(idate.month).zfill(2) + str(idate.day).zfill(2) +\
            file_suff
     else:
       ff = data_path + "/"+\
            file_pref + "0001" + str(idate.month).zfill(2) + str(idate.day).zfill(2) +\
            file_suff

     #print(ff)
     files_to_read.append(ff)
   return files_to_read

# We need to mask out land
def apply_mask( input_field, mask, tol=0.1):
  output_field = np.copy( input_field)
  output_field [mask<tol] = np.nan
  return output_field

# mask land
def mask_array(ds, iTime, vName='SST', mask=my_mask):
    arr = ds[vName].isel(time=iTime).values
    masked_arr=apply_mask(arr, mask)
    return masked_arr

# to write forecast stats
def write_stats(vName, var):
    f1 = vName + "_{}_{}.csv".format(exp_dates[0].strftime('%Y%m%d'), exp_dates[-1].strftime('%Y%m%d'))
    print("Writing out: ", f1)
    np.savetxt(f1, (var), delimiter=",", fmt='%1.4f')
    print("Done!")
    
# Unweighted mean and std dev
def unWeighted_mean_sdev(arr):
    mean_arr = np.nanmean(arr.flatten(), dtype=np.float64)
    sdev_arr =  np.nanstd(arr.flatten(), dtype=np.float64)
    return mean_arr, sdev_arr

### Different methods to predict future BCs, see below for mathematical details

In [ ]:
def forecast_bc(method, id, bc0, clim_bc, anomaly0):
    
    predicted_bc = np.zeros_like(bc0) # init to be safe!
    
    if method == "persist":
        predicted_bc = bc0 # persistence throughout the forecast
    elif method == "persist_init_anom":
        if (id==0):
            predicted_bc = bc0 # forecast start day
        else:
            predicted_bc = clim_bc + anomaly0
    elif method == "test3":
        if (id==0):
            predicted_bc = bc0 # forecast start day
        else:
            predicted_bc = clim_bc - anomaly0
    elif method == "test4":
        if (id==0):
            predicted_bc = bc0 # forecast start day
        else:
            predicted_bc = bc0 + anomaly0       
    else:
        print("Uknown method: {} for creating future BCs.".format(method))
        
    return predicted_bc    

# User set arguments

| variable | What is it? |
| :--      | --:         |
| fcst_nDays | Typical number of days of a single forecast |
| nfcst | How many forecasts should we try |
| vName | Name of Boundary Condition (BC): `SST` or `FRACI` |
| start_date | Start date of ALL the forecasts |
| end_date   | End date of ALL the forecasts |
| data_path_real | Path to `netcdf` version of **daily** GMAO OPS SST and sea ice boundary conditions (BCs)|
| data_path_clim | Path to daily climatology of GMAO OPS BCs, based on 2007- 2023 |

## Methods:
1. **Persist**: Use BC of first day ($x0$) of forecast for all `fcst_nDays`
    $$x(d) = x(0)\,\, \forall d > 0.$$
2. **Persist initial anomaly**: Persist anomaly (with respect to daily clim: $\bar{x}$) of $x(0)$ for for all `fcst_nDays`
    $$\delta x(d) = x(d) - \bar{x}(d) \, \forall d, $$
    $$x(d) = \bar{x}(d) + \delta x(0)\,\, \forall d > 0.$$
3. **Test3**: See below!
    $$x(d) = \bar{x}(d) - \delta x(0).$$
4. **Test4**:
    $$x(d) = x(0) + \delta x(0).$$
5. **Test5**:
    $$x(d) = x(0) + \delta x(-d).$$
    
### Remarks:
- We will _march_ in time: `start_date` -> `end_date` in increments of `fcst_nDays`.
- Above implies `end_date` **must** _fit_ with `nfcst`.
- Check with Kristian Mogensen to be sure above `Persist initial anomaly` was done this way at ECMWF **before** coupled forecasts in IFS.

In [ ]:
fcst_nDays, nfcst = [10, 90]

start_date, end_date = ['2023-06-01', '2023-09-01'] # end_date must fit above.

data_path_real = "/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/GMAO_OPS_bin_data/data/"
data_path_clim = "/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/data/ncFiles/"

file_pref_real, file_suff = ["sst_ice_", ".nc"]
file_pref_clim, file_suff = ["daily_clim_mean_sst_fraci_", ".nc"]

# Select _forecast_ method
#method = "persist" 
method = "persist_init_anom"
#method = "test3"
#method = "test4"

vName = 'SST'

## Dates of all forecasts

In [ ]:
# One forecast per day, since this is daily BCs.
exp_dates  = pd.date_range(start_date, end_date, freq='D')

## Initialize arrays

In [ ]:
mean_spatial_error = np.full((fcst_nDays, my_mask.shape[0], my_mask.shape[1]), 0.0)

# With respect to real data -- remember, we _test_ in **hindcast** mode, so we know the _truth_.
mean_error = np.zeros((fcst_nDays, nfcst), dtype=np.float64)
sdev_error = np.zeros_like( mean_error)

# With respect to daily climatology
mean_error_clim = np.zeros((fcst_nDays, nfcst), dtype=np.float64)
sdev_error_clim = np.zeros_like( mean_error_clim)

if vName == 'FRACI':
    mean_sh_real_error = np.zeros_like(mean_error); sdev_sh_real_error = np.zeros_like(mean_error)
    mean_nh_real_error = np.zeros_like(mean_error); sdev_nh_real_error = np.zeros_like(mean_error)
    
    mean_sh_clim_error = np.zeros_like(mean_error); sdev_sh_clim_error = np.zeros_like(mean_error)
    mean_nh_clim_error = np.zeros_like(mean_error); sdev_nh_clim_error = np.zeros_like(mean_error)

### Compute time-series of global mean and std dev

In [ ]:
for ifcst in range(1, nfcst+1): # each forecast

  fcst_start_date = exp_dates[0] + pd.DateOffset(days=ifcst-1)
  fcst_dates = pd.date_range(start=fcst_start_date, periods=fcst_nDays)
  print("Forecast [{}] Dates: {}".format(ifcst,fcst_dates))

  files_names_real_data = get_files_names(fcst_dates, data_path_real, file_pref_real)
  clim_files_names      = get_files_names(fcst_dates, data_path_clim, file_pref_clim, clim=True)

  ds_real = xr.open_mfdataset(files_names_real_data)
  ds_clim = xr.open_mfdataset(clim_files_names, concat_dim='time', combine='nested', use_cftime=True)

  for id in range(0, fcst_nDays): # over each day of forecast
    if (vName == 'SST'):
        real_bc = mask_array(ds_real, id); clim_bc = mask_array(ds_clim, id)
    else:
        real_bc = mask_array(ds_real, id, vName); clim_bc = mask_array(ds_clim, id, vName)

    # save initial BC (SST/ICE)
    if (id==0):
      bc0 = real_bc; anom0 = bc0 - clim_bc
   
    predicted_bc = forecast_bc(method, id, bc0, clim_bc, anom0)
    
    error_real = predicted_bc - real_bc
    error_clim = clim_bc - real_bc # difference from climatology- if clim was used
        
    mean_error[id,ifcst-1], sdev_error[id, ifcst-1] = unWeighted_mean_sdev(error_real)
    mean_error_clim[id,ifcst-1], sdev_error_clim[id, ifcst-1] = unWeighted_mean_sdev(error_clim)
  
    if vName == 'FRACI':
        sh_real = error_real[0:int(my_mask.shape[0]/2),:]; nh_real = error_real[int(my_mask.shape[0]/2)+1:,:]
        sh_clim = error_clim[0:int(my_mask.shape[0]/2),:]; nh_clim = error_clim[int(my_mask.shape[0]/2)+1:,:]
        
        mean_sh_real_error[id,ifcst-1], sdev_sh_real_error[id, ifcst-1] = unWeighted_mean_sdev(sh_real)
        mean_nh_real_error[id,ifcst-1], sdev_nh_real_error[id, ifcst-1] = unWeighted_mean_sdev(nh_real)
        
        mean_sh_clim_error[id,ifcst-1], sdev_sh_clim_error[id, ifcst-1] = unWeighted_mean_sdev(sh_clim)
        mean_nh_clim_error[id,ifcst-1], sdev_nh_clim_error[id, ifcst-1] = unWeighted_mean_sdev(nh_clim)
    
    if (ifcst ==1): # first forecast
        mean_spatial_error[id,:,:] = error_real
    else:
        mean_spatial_error[id,:,:] = mean_spatial_error[id,:,:] + error_real
#    
for id in range(0, fcst_nDays):
    mean_spatial_error[id,:,:] = mean_spatial_error[id,:,:]/nfcst    

### Plot

In [ ]:
plt.figure( figsize=(12, 4))
plt.subplot(121)
for id in range(0, nfcst):
  plt.plot(range(0, fcst_nDays), mean_error[:,id],      ls='-', c='b', alpha=0.05)
  plt.plot(range(0, fcst_nDays), mean_error_clim[:,id], ls='-', c='k', alpha=0.05)

x1 = np.zeros( (fcst_nDays), dtype=np.float64)
x2 = np.zeros_like(x1)
for id in range(0, fcst_nDays):
    x1[id] = np.mean(mean_error[id,:],  dtype=np.float64)
    x2[id] = np.mean(mean_error_clim[id,:],  dtype=np.float64)
plt.plot(range(0, fcst_nDays), x1, ls='-', c='b', lw=2)
plt.plot(range(0, fcst_nDays), x2, ls='-', c='k', lw=2)

plt.title('Global mean error')
plt.xlabel('Forecast days')
plt.grid(True)
plt.ylim(-0.5, 0.5)
#
plt.subplot(122)
for id in range(0, nfcst):
  plt.plot(range(0, fcst_nDays), sdev_error[:,id],      ls='-', c='b', alpha=0.5)
  plt.plot(range(0, fcst_nDays), sdev_error_clim[:,id], ls='-', c='k', alpha=0.5)
plt.title('SDEV of global mean error')
plt.xlabel('Forecast days')
plt.grid(True)
plt.ylim(0., 1.0)    

In [ ]:
if vName == 'FRACI':
    plt.figure( figsize=(12, 8))
    
    plt.subplot(221)
    for id in range(0, nfcst):
        plt.plot(range(0, fcst_nDays), mean_nh_real_error[:,id], ls='-', c='b', alpha=0.5)
        plt.plot(range(0, fcst_nDays), mean_nh_clim_error[:,id], ls='-', c='k', alpha=0.5)
    plt.title('NH mean (error)')
    plt.grid(True)
    plt.ylim(-0.025, 0.025)
    
    plt.subplot(222)
    for id in range(0, nfcst):
        plt.plot(range(0, fcst_nDays), sdev_nh_real_error[:,id], ls='-', c='b', alpha=0.5)
        plt.plot(range(0, fcst_nDays), sdev_nh_clim_error[:,id], ls='-', c='k', alpha=0.5)
    plt.title('NH SDEV (error)')
    plt.grid(True)
    plt.ylim(0., 0.1)
    
    plt.subplot(223)
    for id in range(0, nfcst):
        plt.plot(range(0, fcst_nDays), mean_sh_real_error[:,id], ls='-', c='b', alpha=0.5)
        plt.plot(range(0, fcst_nDays), mean_sh_clim_error[:,id], ls='-', c='k', alpha=0.5)
    plt.title('SH mean (error)')
    plt.grid(True)
    plt.xlabel('Forecast days')
    plt.ylim(-0.025, 0.025)
    
    plt.subplot(224)
    for id in range(0, nfcst):
        plt.plot(range(0, fcst_nDays), sdev_sh_real_error[:,id], ls='-', c='b', alpha=0.5)
        plt.plot(range(0, fcst_nDays), sdev_sh_clim_error[:,id], ls='-', c='k', alpha=0.5)
    plt.title('SH SDEV (error)')
    plt.xlabel('Forecast days')
    plt.grid(True)
    plt.ylim(0., 0.1) 

### Calculate global mean in two different ways to make sure spatial mean is correct

In [ ]:
print("Forecast Day\tGlobal Mean Error (time-series)\tFrom spatial maps\t Difference between the two methods\n")

for id in range(0, fcst_nDays):
    x1 = np.nanmean( mean_spatial_error[id,:,:], dtype=np.float64)
    x2 = np.mean(mean_error[id,:], dtype=np.float64)
    print(id+1,"\t\t",x1,"\t\t",x2,"\t",x2-x1)
    
print("\n--> If both methods give same answer, (last column == 0), all good!")    

In [ ]:
if vName == 'SST':
    cMin, cMax = [-1., 1.]
else:
    cMin, cMax = [-.2, .2]

plt.figure( figsize=(16, 10))

for id in range(0, fcst_nDays):
    plt.subplot(4, 3, id+1)
    plt.pcolormesh(land_sea_mask.lon.values, land_sea_mask.lat.values,\
                   mean_spatial_error[id,:,:], vmin=cMin, vmax=cMax, cmap=plt.cm.bwr)
    plt.colorbar()
    plt.title('day: {}'.format(id))

## Save processed data (for later use)

In [ ]:
write_stats('mean_{}_error_'.format(vName) + method, mean_error)
write_stats('sdev_{}_error_'.format(vName) + method, sdev_error)

write_stats('mean_{}_error_clim_'.format(vName) + method, mean_error_clim)
write_stats('sdev_{}_error_clim_'.format(vName) + method, sdev_error_clim)

if vName == 'FRACI':
    write_stats('mean_{}_nh_real_error_'.format(vName) + method, mean_nh_real_error)
    write_stats('sdev_{}_nh_real_error_'.format(vName) + method, sdev_nh_real_error)
    
    write_stats('mean_{}_sh_real_error_'.format(vName) + method, mean_sh_real_error)
    write_stats('sdev_{}_sh_real_error_'.format(vName) + method, sdev_sh_real_error)
    
    write_stats('mean_{}_nh_clim_error_'.format(vName) + method, mean_nh_clim_error)
    write_stats('sdev_{}_nh_clim_error_'.format(vName) + method, sdev_nh_clim_error)
    
    write_stats('mean_{}_sh_clim_error_'.format(vName) + method, mean_sh_clim_error)
    write_stats('sdev_{}_sh_clim_error_'.format(vName) + method, sdev_sh_clim_error)

In [ ]:
da = xr.DataArray(data=mean_spatial_error, 
coords={'time': np.arange(0, fcst_nDays), 'lat': land_sea_mask.lat,'lon': land_sea_mask.lon}, 
dims=["time", "lat", "lon"],
attrs=dict(description="Mean of forecast error in BCs, see https://github.com/sanAkel/future_sst_fraci/blob/main/to_gen_new_files/SST_ideas.ipynb"))

ds_bc_err = da.to_dataset(name='{}_fcst_error'.format(vName))

# vName = SST, FRACI
#ds_bc_err.SST_fcst_error.plot(x="lon", y="lat", col="time", col_wrap=3, vmin=cMin, vmax=cMax, cmap=plt.cm.bwr)
#ds_bc_err.FRACI_fcst_error.plot(x="lon", y="lat", col="time", col_wrap=3, vmin=cMin, vmax=cMax, cmap=plt.cm.bwr)

ds_bc_err.to_netcdf("{}_mean_err_{}_{}_{}.nc".format(vName, method, exp_dates[0].strftime('%Y%m%d'), exp_dates[-1].strftime('%Y%m%d')))

## Compute std dev for each of the `fcst_nDays` using all `nfcst` forecasts

In [ ]:
# Initialize
sdev_spatial_error = np.zeros_like(mean_spatial_error)
sum_sq_spatial_error = np.zeros_like(mean_spatial_error)

In [ ]:
for ifcst in range(1, nfcst+1): # each forecast

  fcst_start_date = exp_dates[0] + pd.DateOffset(days=ifcst-1)
  fcst_dates = pd.date_range(start=fcst_start_date, periods=fcst_nDays)
  print("Forecast [{}] Dates: {}".format(ifcst,fcst_dates))

  files_names_real_data = get_files_names(fcst_dates, data_path_real, file_pref_real)
  clim_files_names      = get_files_names(fcst_dates, data_path_clim, file_pref_clim, clim=True)

  ds_real = xr.open_mfdataset(files_names_real_data)
  ds_clim = xr.open_mfdataset(clim_files_names, concat_dim='time', combine='nested', use_cftime=True)

  for id in range(0, fcst_nDays): # over each day of forecast
    if (vName == 'SST'):
        real_bc = mask_array(ds_real, id); clim_bc = mask_array(ds_clim, id)
    else:
        real_bc = mask_array(ds_real, id, vName); clim_bc = mask_array(ds_clim, id, vName)

    # save initial BC (SST/ICE)
    if (id==0):
      bc0 = real_bc; anom0 = bc0 - clim_bc
   
    predicted_bc = forecast_bc(method, id, bc0, clim_bc, anom0) 
    error_real = predicted_bc - real_bc
    
    if (ifcst ==1): # first forecast
        sum_sq_spatial_error[id,:,:] = (error_real-mean_spatial_error[id,:,:])**2.
    else:
        sum_sq_spatial_error[id,:,:] = sum_sq_spatial_error[id,:,:] + (error_real-mean_spatial_error[id,:,:])**2.
#    
for id in range(0, fcst_nDays):
    sdev_spatial_error[id,:,:] = np.sqrt(sum_sq_spatial_error[id,:,:]/(nfcst-1)) # sample (n-1), not population (n).  

In [ ]:
if vName == 'SST':
    cMin, cMax = [0.05, 2.]
else:
    cMin, cMax = [0.01, .5]

plt.figure( figsize=(16, 10))

for id in range(0, fcst_nDays):
    plt.subplot(4, 3, id+1)
    plt.pcolormesh(land_sea_mask.lon.values, land_sea_mask.lat.values,\
                   sdev_spatial_error[id,:,:], vmin=cMin, vmax=cMax, cmap=plt.cm.Spectral_r)
    plt.colorbar()
    plt.title('day: {}'.format(id))

In [ ]:
da = xr.DataArray(data=sdev_spatial_error, 
coords={'time': np.arange(0, fcst_nDays), 'lat': land_sea_mask.lat,'lon': land_sea_mask.lon}, 
dims=["time", "lat", "lon"],
attrs=dict(description="Standard deviation of forecast error in BCs, see https://github.com/sanAkel/future_sst_fraci/blob/main/to_gen_new_files/SST_ideas.ipynb"))

ds_bc_err = da.to_dataset(name='{}_fcst_error'.format(vName))

# vName = SST, FRACI
#ds_bc_err.SST_fcst_error.plot(x="lon", y="lat", col="time", col_wrap=3, vmin=cMin, vmax=cMax, cmap=plt.cm.Spectral_r)
#ds_bc_err.FRACI_fcst_error.plot(x="lon", y="lat", col="time", col_wrap=3, vmin=cMin, vmax=cMax, cmap=plt.cm.Spectral_r)

ds_bc_err.to_netcdf("{}_sdev_err_{}_{}_{}.nc".format(vName, method, exp_dates[0].strftime('%Y%m%d'), exp_dates[-1].strftime('%Y%m%d')))